In [ ]:
from google.colab import drive
!pip install joblib
!pip install colorama
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from keras.layers import SimpleRNN,Dense
from keras.utils import to_categorical
import joblib
from colorama import Fore,Back,Style


drive.mount('/content/drive')
model_yolu='/content/drive/My Drive/AI_Bootcamp/rnn.h5'
veri_yol="/content/drive/MyDrive/Colab Notebooks/Phishing_Email.csv"
print(veri_yol)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Phishing_Email.csv


In [ ]:
mail_verisi = pd.read_csv(veri_yol)
mail_verisi = mail_verisi.where((pd.notnull(mail_verisi)), '')

mail_verisi.loc[mail_verisi['Email Type'] =='Safe Email','Email Type'] = 0
mail_verisi.loc[mail_verisi['Email Type'] =='Phishing Email','Email Type'] = 1


#Tokenizer

In [ ]:
x = mail_verisi['Email Text']
y = mail_verisi['Email Type']


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
indeksli_veri = tokenizer.texts_to_sequences(x)

word_index = tokenizer.word_index
kelime_uzunluğu = 100
sabit_uzunluk = pad_sequences(indeksli_veri,maxlen=kelime_uzunluğu)


In [ ]:
y = to_categorical(y)
print("Kelime İndexleri", word_index)



#RNN

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(sabit_uzunluk,y,test_size=0.2,random_state=1)


In [ ]:
from tensorflow.keras.layers import Embedding
model=Sequential()
model.add(Embedding(len(word_index)+1,output_dim=128,input_length=kelime_uzunluğu))
model.add(SimpleRNN(64,activation="relu"))
model.add(Dense(2,activation="softmax"))
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.fit(X_train,y_train,epochs=5,batch_size=32,validation_data=(X_test,y_test))


Epoch 1/5
467/467 ━━━━━━━━━━━━━━━━━━━━ 175s 366ms/step - accuracy: 0.7704 - loss: 0.4347 - val_accuracy: 0.9523 - val_loss: 0.1677
Epoch 2/5
467/467 ━━━━━━━━━━━━━━━━━━━━ 210s 383ms/step - accuracy: 0.9744 - loss: 0.0785 - val_accuracy: 0.9523 - val_loss: 0.1126
Epoch 3/5
467/467 ━━━━━━━━━━━━━━━━━━━━ 173s 371ms/step - accuracy: 0.9800 - loss: 0.0502 - val_accuracy: 0.9552 - val_loss: 0.1456
Epoch 4/5
467/467 ━━━━━━━━━━━━━━━━━━━━ 201s 369ms/step - accuracy: 0.9837 - loss: 0.0385 - val_accuracy: 0.9217 - val_loss: 0.1978
Epoch 5/5
467/467 ━━━━━━━━━━━━━━━━━━━━ 184s 393ms/step - accuracy: 0.9841 - loss: 0.0315 - val_accuracy: 0.9450 - val_loss: 0.1492


In [ ]:
model.save(model_yolu)


In [ ]:
import numpy as np
while True:
  giris= input("maili girin")
  sayısal_kelime=tokenizer.texts_to_sequences([giris])
  sabit_uzunluk = pad_sequences(sayısal_kelime, maxlen=kelime_uzunluğu)
  tahmin = model.predict(sabit_uzunluk)
  if np.argmax(tahmin)==1:
    print(Fore.RED, 'Bu bir kimlik avı emaili olabilir')
  else:
    print(Fore.GREEN, 'Bu güvenli emailidir.')
    print(Fore.WHITE, 'Doğruluk puanı:', tahmin[0][np.argmax(tahmin)])
  break



maili girinre : meeting on feb 8 , 2001 dear mr . nur azmin abu bakar , thanks for your prompt reply . please , let us know how many members of your team will visit enron . i look forward to our meeting on february 8 . vince kaminski azminab @ petronas . com . my on 01 / 02 / 2001 06 : 38 : 33 pm to : vince . j . kaminski @ enron . com , khairuddinbmjaafar @ petronas . com . my , shirley . crenshaw @ enron . com cc : subject : re : meeting on feb 8 , 2001 dear kaminski , happy new year and thank you for the reply . we are honored to have lunch with you and your team however we have another appointment at 2 . 30 p . m . regards vince . j . kaminski @ enron . com on 03 / 01 / 2001 07 : 38 : 19 am to : azminab @ petronas . com . my cc : vince . j . kaminski @ enron . com , shirley . crenshaw @ enron . com subject : meeting on feb 8 , 2001 dear sir , i would like to apologize for the delay in responding to your fax . i was on vacation for the last few days . i shall be honored to meet your